In [9]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file 
SERPAPI_API_KEY = os.environ['SERPAPI_API_KEY']



if not SERPAPI_API_KEY:  
  raise ValueError("Please set SERPAPI_API_KEY environment variables") 


In [5]:
from langchain_ollama import ChatOllama 
from langchain_community.utilities import SerpAPIWrapper 

# Initialize LLM 
llm = ChatOllama(model="llama2", temperature=0) 

# Web search tool using SerpAPI 
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY) 


C:\Users\Drishti_De\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from typing import TypedDict 
  
class ResearchState(TypedDict, total=False): 
  query: str 
  search_results: str 
  summary: str 
  store_message: str 
  response: str 


def web_search_tool(query: str) -> str:  
  print(f"Searching the web for: {query}")  
  results = search.run(query)  
  return results 

# ------------------------------

from langchain.chains import LLMChain 
from langchain.prompts import PromptTemplate 

def summarize_tool(text: str) -> str: 
  # Basic prompt to summarize text 
  prompt = PromptTemplate( 
    input_variables=["text"], 
    template="Summarize the following text in a concise way:\n\n{text}" 
  ) 

  chain = prompt | llm 
  result = chain.invoke({"text": text}) 
  return result.content if hasattr(result, "content") else str(result) 


memory_store = {} 

def store_tool(data: dict) -> str: 
  query = data.get("query") 
  summary = data.get("summary") 
  if query and summary: 
    memory_store[query] = summary 
    return f"Stored summary for query: '{query}'" 
  return "No data to store" 

def respond_tool(query: str) -> str: 
  summary = memory_store.get(query) 
  return f"Final Answer:\n{summary}" if summary else "Sorry, no summary found for your query." 

def ask_handler(state): 
  user_query = input("What do you want to research? ") 
  state["query"] = user_query 
  return state 

def search_handler(state): 
  query = state.get("query", "") 
  state["search_results"] = web_search_tool(query) 
  return state 

def summarize_handler(state): 
  state["summary"] = summarize_tool(state.get("search_results", "")) 
  return state 

def store_handler(state): 
  state["store_message"] = store_tool({ 
    "query": state.get("query"), 
    "summary": state.get("summary") 
  }) 
  return state 
 
def respond_handler(state): 
  state["response"] = respond_tool(state.get("query")) 
  return state 


In [7]:
from langgraph.graph.state import StateGraph 

graph = StateGraph(state_schema=ResearchState) 

graph.add_node("ask", ask_handler)  
graph.add_node("search", search_handler)  
graph.add_node("summarize", summarize_handler)  
graph.add_node("store", store_handler)  
graph.add_node("respond", respond_handler) 

graph.set_entry_point("ask")  

graph.add_edge("ask", "search")  
graph.add_edge("search", "summarize")  
graph.add_edge("summarize", "store")  
graph.add_edge("store", "respond") 

research_agent = graph.compile()


In [8]:
state = {}  
final_state = research_agent.invoke(state)  
print(final_state.get("response")) 


Searching the web for: What is the impact of Agentic AI in our world?
Final Answer:
Agentic AI is a significant shift in the relationship between humans and intelligent systems, enabling AI agents to operate autonomously, solve complex tasks, and adapt in real-time without continuous human guidance. Agentic AI interprets data, learns from interactions, and makes decisions independently, redeploying the workforce and shifting workers to supervising AI agents. This technology is becoming more widely used in decision-making, automating complex business processes and ensuring impeccable data validation in real-time.
